**Name:** Sarah Sexton-Bowser <br/>
**Semester:** Spring 2020 <br/>
**Project Objective:** Analysis yield response of sorghum and corn to precipitation during water deficit growing seasons for Kansas cropping districts. </br>
**Code Objective**: To process crop yield and weather data for the NW Kansas crop reporting district.  

In [2]:
#Import Module
import pandas as pd

# Load and Set-up the Data

**Processing of Data:** Data was downloaded direct from source. Any manipulation of data is done in python. 
1. Data for yield of sorghum and corn is obtained by USDA Quick Stats from USDA NASS. https://quickstats.nass.usda.gov/ 
2. Data for weather variables is obtained by NOAA. https://www7.ncdc.noaa.gov/CDO/CDODivisionalSelect.jsp  

**Load NASS Data for Crop Yield Variables.**  

In [3]:
all_district = pd.read_csv('C:/code/sorghum_acres_prediction/planting_price_NASS_1949_2019_all_districts.csv') # read file
all_district = all_district.rename(columns={'Ag District':'District'}) # rename column to delete space for future calling
NW = all_district.District.isin(['NORTHWEST']) # index all cells with district of interest. This code can be reproduced for other districts.
nass = all_district[NW].loc[:] # create dataframe with district of interest
nass = nass.rename(columns={'Data Item': 'Data'}) #rename column to delete space for future calling
nass.head(5)

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data,Domain,Domain Category,Value,CV (%)
16,SURVEY,2019,YEAR,NaN,AGRICULTURAL DISTRICT,KANSAS,20,NORTHWEST,10,NaN,...,NaN,NaN,0,NaN,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,"1,294,000",NaN
17,SURVEY,2019,YEAR,NaN,AGRICULTURAL DISTRICT,KANSAS,20,NORTHWEST,10,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,127.7,NaN
18,SURVEY,2019,YEAR,NaN,AGRICULTURAL DISTRICT,KANSAS,20,NORTHWEST,10,NaN,...,NaN,NaN,0,NaN,SORGHUM,SORGHUM - ACRES PLANTED,TOTAL,NOT SPECIFIED,"172,000",NaN
19,SURVEY,2019,YEAR,NaN,AGRICULTURAL DISTRICT,KANSAS,20,NORTHWEST,10,NaN,...,NaN,NaN,0,NaN,SORGHUM,"SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,88.5,NaN
56,SURVEY,2018,YEAR,NaN,AGRICULTURAL DISTRICT,KANSAS,20,NORTHWEST,10,NaN,...,NaN,NaN,0,NaN,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,"1,116,000",NaN


**Load NOAA Data for Weather Variables.**

In [4]:
# Load NOAA Data for Weather Variable. 
weather_data = pd.read_csv('C:/code/sorghum_acres_prediction/noaa_allmonth_1949_2019_alldistricts.txt', sep=",") 

''' Division codes 
01 is NW, 02 is NC, 03 is NE, 04 is WC, 05 is C, 
06 is EC, 07 is SW, 08 is SC, 09 is SE
'''
NW = weather_data.Division.isin(['01']) #index all cells for NW/Division 07
weather_data = weather_data[NW].loc[:] #create dataframe with district of interest
weather_data.head(5)

,StateCode,Division,YearMonth,PCP,TAVG,PDSI,PHDI,ZNDX,PMDI,CDD,...,SP01,SP02,SP03,SP06,SP09,SP12,SP24,TMIN,TMAX,Unnamed: 20
2,14,1,194901,0.84,16.2,1.67,1.67,1.12,1.48,0,...,1.32,0.84,1.29,-0.16,0.42,0.12,-0.12,5.8,26.6,NaN
17,14,1,194902,0.53,29.0,1.60,1.60,0.33,1.50,0,...,0.20,0.89,0.67,-0.21,0.40,0.13,-0.07,16.4,41.4,NaN
20,14,1,194903,2.63,38.3,2.29,2.29,2.56,2.29,0,...,1.48,1.33,1.50,0.96,0.25,0.36,0.16,26.8,49.7,NaN
31,14,1,194904,1.78,50.2,2.21,2.21,0.46,2.21,0,...,-0.03,0.88,0.83,1.20,0.30,0.65,0.15,36.7,63.8,NaN
37,14,1,194905,5.27,61.9,3.10,3.10,3.36,3.10,59,...,1.28,1.00,1.46,1.51,0.98,1.17,0.52,50.4,73.4,NaN


**Summary of manipulation of NOAA data to create variables of drought index, and preciptation by month and by growing season** </br>
1. Change 'YearMonth' to datetime and create columns for month and year 
2. Change column names and types for compatability with processing
3. Create an annual calendar year precipitation variable 
4. Create monthly variables for precipitation and drought

In [5]:
# Change NOAA Data variables YearMonth to string, and add columns with just year and just month for subsequent processing 
weather_data["timestamp"] = pd.to_datetime(weather_data["YearMonth"], format="%Y%m") # datetime from YearMonth
weather_data['Year'] = weather_data["timestamp"].dt.year # create column for year
weather_data['Month'] = weather_data["timestamp"].dt.month # create column for month
weather_data['PCP'] = weather_data['    PCP'].astype(float) # change type for processing
weather_data['PDSI'] = weather_data['   PDSI'].astype(float) # change type for processing

# Splice to year and precip variables and sum precip for all months in each year
years = weather_data.loc[0: , ['Year','PCP']] # identify all years
annual_prec = years.groupby('Year').sum()  # add all months of precipitation for each year
annual_prec = annual_prec.iloc[0:71] 
annual_prec = annual_prec.reset_index() # made year a column so it is callable to merge

# Splice to month precip variables 
months = weather_data.loc[0: , ['Month','PCP', 'PDSI','Year']] # identify all months

In [6]:
# Create variable of drought and precipitation for each month

# Function to process extracting the precipitation and drought index for each month
def getMonth (data, element):
    month_variable = data.Month.isin([element]) # Source: Fixed by AP with removal of "" around element 
    month_variable = data[month_variable].loc[0: , ['Year', 'PCP', 'PDSI']]
    month_variable = month_variable.rename(columns={'PCP': 'PCP'+element, 'PDSI':'PDSI'+ element}) 
    return month_variable 

# Ran each month in the getMonths function
January = getMonth(months,'1')
February = getMonth(months,'2')
March = getMonth(months,'3')
April = getMonth(months,'4')
May = getMonth(months,'5')
June = getMonth(months,'6')
July = getMonth(months,'7')
August = getMonth(months,'8')
September = getMonth(months,'9')
October = getMonth(months,'10')
November = getMonth(months,'11')
December = getMonth(months,'12')

# Merged for a single dataframe with all months 
month_index = (pd.merge(left=January, right=February, on='Year').merge(right=March, on='Year').merge(right=April, on='Year').merge(right=May, on='Year')
                .merge(right=June, on='Year').merge(right=July, on='Year').merge(right=August, on='Year').merge(right=September, on='Year')
                .merge(right=October, on='Year').merge(right=November, on='Year').merge(right=December, on='Year'))

**Summary of manipulation of NASS data to create variables of yield** </br>
1. Identify corn and sorghum for respective values in the rows
2. Change column names and types for compatability with processing

In [7]:
#Corn Yield Variable
corn_yield = nass.Data.isin(['CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE']) # identify corn yield data 
corn_yield = nass[corn_yield].loc[0: , ['Year','Value']] # extract data for year and value  
corn_yield=corn_yield.rename(columns={'Value': 'Corn_Yield'}).astype({'Corn_Yield': 'float32'}) # Change data type to float for mathmatical operations 

#Sorghum Yield Variable
sorghum_yield = nass.Data.isin(['SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE']) # identify sorghum yield data 
sorghum_yield = nass[sorghum_yield].loc[0: , ['Year','Value']] # extract data for year and value  
sorghum_yield=sorghum_yield.rename(columns={'Value': 'Sor_Yield'}).astype({'Sor_Yield': 'float32'}) # Change data type to float for mathmatical operations 

**Create dataframe will all variables for weather and crop yield.** Data is merged on the year.

In [8]:
# Merge data frames created in prior cell with use of year as key to append 
df = pd.merge(left=sorghum_yield, right=corn_yield, on='Year').merge(right=month_index, on='Year').merge(right=annual_prec, on='Year')
df.head()

,Year,Sor_Yield,Corn_Yield,PCP1,PDSI1,PCP2,PDSI2,PCP3,PDSI3,PCP4,...,PDSI8,PCP9,PDSI9,PCP10,PDSI10,PCP11,PDSI11,PCP12,PDSI12,PCP
0,2018,91.599998,106.500000,0.63,-0.55,0.42,-0.53,0.59,-1.03,1.09,...,0.34,1.62,0.11,3.31,1.53,0.59,1.43,1.63,2.34,24.61
1,2017,81.500000,107.800003,1.03,0.46,0.05,-0.57,2.11,0.06,2.25,...,1.58,2.95,2.17,1.78,2.40,0.10,-0.54,0.09,-0.78,24.55
2,2016,91.900002,103.400002,0.12,-0.25,0.87,-0.06,0.39,-0.84,5.59,...,0.75,3.52,1.69,0.05,-0.79,0.33,-1.25,0.47,-0.01,21.07
3,2015,76.800003,77.599998,0.19,-1.86,0.44,-1.77,0.13,-2.62,2.80,...,-0.63,1.13,-1.20,1.72,0.19,2.28,1.13,0.53,1.06,22.49
4,2014,71.300003,71.099998,0.35,-2.73,0.49,-2.52,0.25,-3.04,1.38,...,-2.16,1.82,-2.06,0.59,-2.35,0.03,-2.59,1.18,-1.78,20.17


# Palmer Drought Score Index (PSDI value)
There are a few values in the magnitude of +7 or -7.  <br/>
PDSI values <br/>
0 to -.5 = normal;<br/>
-0.5 to -1.0 = incipient drought; <br/>
-1.0 to -2.0 = mild drought; <br/>
-2.0 to -3.0 = moderate drought; <br/>
-3.0 to -4.0 = severe drought; <br/>
and greater than -4.0 = extreme drought.   <br/>

The PSDI index is based on the Thornthwaite equation to calcualte potential evaporation based on temperature and time. NOAA uses the potential transportation equation under limits of actual available preciptation. The index accounts for carry over or deficit moisture from prior months for indications of climatic periods of deficit or excess demands for precipitation. 

The Thornthwaite equation is the difference between the aridity index $I_a$ and the humidity index $I_h$. Where PE is potential evaporation $D$ is moisture deficit and $R$ is excess moisture supply or runoff. <br/> 
$I_a = 100(D/PE)$ <br/>
$I_a = 100(R/PE)$ <br/>

$TMI = I_h -0.61_\alpha$ <br/>

Learn more about NOAA's methodology here https://www.ncdc.noaa.gov/monitoring-references/dyk/potential-evapotranspiration 


**Identify years of an excess or deficit moisture during the growing season** 

Each month of June, July, August and September are evaluated and counted if the Palmer Drought Score Index denotes the month as excess or deficit moisture demand. If two or more months are designated excess moisture the year is identified as a year of excess moisture during the summer. If two or more months are designated as decific moisture supply for demand, the year is counted as a moisture deficit year. 

1. Months considered are June, July, August and September
2. Count the month if deficit or excess moisture per the PSDI index
3. Create a dataframe frame for years of excess moisture for at least two or more of the four summer months

In [11]:
#Deficit Moisture Year in Summer Season
D06 = df.PDSI6 < -0.5
D07 = df.PDSI7 < -0.5
D08 = df.PDSI8 < -0.5
D09 = df.PDSI9 < -0.5

DCount = pd.concat([D06,D07,D08,D09],axis=1) #concat each of the booleans for June-Sept
df['DCount'] = DCount.sum(axis=1) #add the number of summer months with deficit moisture 
Didx = df.DCount >=2 #boolean index of two or more months
DFdf = df.loc[Didx] # dataframe with years that have at least 2 or more summer months of deficit moisture 

#Excess Moisture Year in Summer Season
W06 = df.PDSI6 > 0
W07 = df.PDSI7 > 0
W08 = df.PDSI8 > 0
W09 = df.PDSI9 > 0

WCount = pd.concat([W06,W07,W08,W09],axis=1) #concat each of the booleans for June-Sept
df['WCount'] = WCount.sum(axis=1) #add the number of summer months with excess moisture
Widx = df.WCount >=2 #boolean index of two or more months
Wdf = df.loc[Widx] # dataframe with years that have at least 2 or more summer months of excess moisture 

Wdf.head(5)

,Year,Sor_Yield,Corn_Yield,PCP1,PDSI1,PCP2,PDSI2,PCP3,PDSI3,PCP4,...,PDSI9,PCP10,PDSI10,PCP11,PDSI11,PCP12,PDSI12,PCP,DCount,WCount
0,2018,91.599998,106.500000,0.63,-0.55,0.42,-0.53,0.59,-1.03,1.09,...,0.11,3.31,1.53,0.59,1.43,1.63,2.34,24.61,0,4
1,2017,81.500000,107.800003,1.03,0.46,0.05,-0.57,2.11,0.06,2.25,...,2.17,1.78,2.40,0.10,-0.54,0.09,-0.78,24.55,0,4
2,2016,91.900002,103.400002,0.12,-0.25,0.87,-0.06,0.39,-0.84,5.59,...,1.69,0.05,-0.79,0.33,-1.25,0.47,-0.01,21.07,0,4
7,2011,76.300003,78.000000,0.37,-0.72,0.50,-0.61,0.67,-0.95,2.07,...,0.64,3.05,1.71,0.45,1.42,0.61,1.44,22.93,1,3
8,2010,81.000000,83.400002,0.12,4.84,0.41,4.53,2.02,4.70,3.36,...,4.71,0.32,-0.48,0.37,-0.66,0.24,-0.75,19.60,0,4


**Export a csv for the NW crop reporting district**
1. NWdf as the entire set of NW data
2. NWDF as NW deficit moisture data
3. NWWDf as NW excess moisture data

In [19]:
df.to_csv('NWdf.csv', index=False) # entire dataset
DFdf.to_csv('NWDFdf.csv', index=False) # summers with deficit moisture
Wdf.to_csv('NWWdf.csv', index=False) # summers with excess moisture 

**Sources** <br/>
McCabe Jr, G. J., Wolock, D. M., Hay, L. E., & Ayers, M. A. (1990). EFFECTS OF CLIMATIC CHANGE ON THE THORNTHWAITE MOISTURE INDEX 1. JAWRA Journal of the American Water Resources Association, 26(4), 633-643. <br/>

Thornthwaite, C.W. (1948) An Approach toward a Rational Classification of Climate. Geographical Review, 38, 55-94.